In [ ]:
import os
import requests
import sys
import urllib.request
from urllib.request import urlopen
import re
from pandas import DataFrame
import pandas as pd
from pandas.compat import StringIO
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import subprocess
from datetime import datetime
import ipywidgets as widgets
files = [i for i in os.listdir('.') if re.search('.fasta|.fa', i)]
fastas = widgets.Dropdown(options =  files)

In [ ]:
modulo = urllib.request.urlretrieve('https://raw.githubusercontent.com/eduardo1011/Bioinformatica2019/master/blast_px_module.py', 'blast_px_module.py')
import blast_px_module
from blast_px_module import *

In [ ]:
fastas

## Elegimos cuántas secuencias queremos analizar

In [ ]:
fasta = open(fastas.value,'r')
fasta = fasta.read()
print('Número de secuencias:',len(re.findall('>', fasta)))
secs = input('Número de secuencias a analizar (ejemplo: 500): ')
# crear un archivo con 1000 secuencias para el blastx
cons = open(secs+'secuencias.fasta','w')
cons.write('>'.join(fasta.split('>')[0:int(float(secs)) + 1]))
cons.close()
filename = secs+'secuencias.fasta'
print('Archivo nuevo: ', filename)

## Definimos la localización de la base de datos para hacer Blast

In [ ]:
# en este caso la localizacón es:
loc_db = 'DB_A_brasiliensis/DB_A_brasiliensis.fasta'

# 2 Blast[p/x]
### funció para correr blast[x/p]
### <font color=red>run_blast_jupyter(x_p = " ", file = " ", db = " ")</font> 

### Corriendo blast en jupyter

In [ ]:
tipo = 'blastp'

In [ ]:
run_blast_jupyter(x_p = tipo, file = filename, db = loc_db, evalue = 1E-6,
                 max_target_seqs=4, max_hsps=4)

# Nuevas columnas
## <span style="color:red">qstart</span> means Start of alignment in query
## <span style="color:red">qend</span> means End of alignment in query
## <span style="color:red">sstart</span> means Start of alignment in subject
## <span style="color:red">send</span> means End of alignment in subject

## Abrimos el resuado del Blast

In [ ]:
out_file = filename.split('.')[0]
out_file = tipo+'_'+out_file.split('/')[-1]+'.tab'
out_file

In [ ]:
blast_res = pd.read_csv(out_file, sep = '\t')#.dropna()
blast_res

In [ ]:
#aplicar filtro 70% de identidad
#filtro_70 = blast_res[(blast_res.pident >= 70) & (blast_res.pident <= 100)].reset_index(drop=True).drop_duplicates()

In [ ]:
dfs = []
for i in blast_res.qacc.drop_duplicates():
    df = blast_res[blast_res.qacc == i].sort_values(by='bitscore', ascending=False)
    dfs.append(df[:1])
best_hits = pd.concat(dfs)
# descarga las seceuncias filtradas
best_hits.to_csv(out_file.split('.')[0]+'_best_hist.tsv', sep = '\t', index = None)

In [ ]:
best_hits

<img src="https://raw.githubusercontent.com/eduardo1011/Bioinformatica2019/master/blactx-p.png" width = 100%> <br>

In [ ]:
best_hits[['Entry']].drop_duplicates().Entry.count()

In [ ]:
archivo_de_salida1 = filename.split('.')[0]+'.txt'
archivo_de_salida1

### comando para salvar la lista de identificadores

In [ ]:
best_hits[['Entry']].drop_duplicates().to_csv(archivo_de_salida1, sep = '\t', index = None, header = None)

<img src="https://raw.githubusercontent.com/eduardo1011/Bioinformatica2019/master/blactn.png" width = 100%> <br>

In [ ]:
archivo_de_salida2 = out_file.split('.')[0]+'.tsv'
archivo_de_salida2

In [ ]:
file_for_tax = best_hist[['qacc', 'Entry', 'Tax_ID']].drop_duplicates()
file_for_tax.to_csv(archivo_de_salida2, sep = '\t', index = None)